In [1]:
pip install pandas

In [2]:
import pandas as pd

# Load your data from an Excel file (adjust the file name as needed)
file_path = 'your_data_file.xlsx'  # Replace with your file path
data = pd.read_excel(file_path)

# Combine 'Local Date' and 'Local Time' into a single datetime column
data['timestamp'] = pd.to_datetime(data['Local Date'] + ' ' + data['Local Time'], format='%d-%b-%Y %H:%M')

# Set the timestamp as the index and drop the original date and time columns
data.set_index('timestamp', inplace=True)
data.drop(columns=['Local Date', 'Local Time'], inplace=True)

# Sort the data by timestamp (if not already sorted)
data.sort_index(inplace=True)

# Strategy parameters
initial_price = 90000  # Starting price
price_step = 0.01      # 1% steps for buy orders
lots = 0
positions = []         # Track all open positions and their take-profits
profits = 0            # Track total profits from successful take-profits

# Function to place a buy and corresponding take-profit order
def place_order(entry_price):
    global lots
    take_profit_price = entry_price * (1 + price_step)  # 1% above entry price
    positions.append({'entry_price': entry_price, 'take_profit': take_profit_price})
    lots += 1

# Initialize with the first buy
place_order(initial_price)

# Loop through the historical data
for index, row in data.iterrows():
    current_price = row['Close']  # Use the 'Close' price for each 30-min candle

    # Check if any take-profit orders are hit
    for pos in positions.copy():
        if current_price >= pos['take_profit']:
            # Take profit hit, record profit and remove position
            profit = pos['take_profit'] - pos['entry_price']
            profits += profit
            positions.remove(pos)
            # Re-enter the market by placing another buy at the same price
            place_order(pos['entry_price'])

    # Check if price has dropped 1% below the last entry price and place a new buy order
    last_position = positions[-1]  # Last buy price
    if current_price <= last_position['entry_price'] * (1 - price_step):
        new_entry_price = last_position['entry_price'] * (1 - price_step)
        place_order(new_entry_price)

# At the end of the backtest, calculate unrealized loss for remaining positions
unrealized_loss = 0
current_price = data.iloc[-1]['Close']  # Final close price
for pos in positions:
    unrealized_loss += (pos['entry_price'] - current_price)

# Output results
print(f"Total Profit: {profits}")
print(f"Unrealized Loss: {unrealized_loss}")
print(f"Number of Lots Held: {lots}")


FileNotFoundError: [Errno 2] No such file or directory: 'silver_mcx_30min.csv'